# Neural network Classifier x Toxic Content Detection
Il presente Notebook mostra l'addestramento ed il testing di un Classificatore basato su Neural network per il task di Toxic Content Detection.

I dati sono stati processati come segue:
1. Pulizia del testo (si veda, 'dataset_preprocessing.py')
2. Lemmatizzazione con NLTK
3. Vettorizzazione con TF-IDF

In [2]:
import pandas as pd
import pickle
import nltk
import re
import numpy as np
from nltk.tokenize import word_tokenize
from nltk.corpus import wordnet
from nltk.stem import WordNetLemmatizer
from sklearn.feature_extraction.text import TfidfVectorizer
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from keras.layers import Conv1D
from keras.layers import MaxPooling1D
from sklearn.decomposition import PCA
from datetime import datetime
from sklearn.metrics import accuracy_score, precision_score, recall_score, confusion_matrix, ConfusionMatrixDisplay

/var/folders/ss/s59pk8px01vb8p_b48z9wxz40000gn/T/ipykernel_11020/2817353859.py:1: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was not found to be installed on your system.
If this would cause problems for you,
please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466
        
  import pandas as pd


# Neural Network, Dataset "non-Lemmatizzato"

In [3]:
training_set = pd.read_csv("./../../datasets/training_set.csv")
test_data = pd.read_csv("./../../datasets/test_set.csv")
test_data.dropna(inplace=True)
test_set = test_data[test_data['toxic']!=-1]
# Osservazione: il Training Set è stato già ripulito

In [4]:
# Vettorizzazione con TF-IDF
vectorizer = TfidfVectorizer()
vectorizer_lem = TfidfVectorizer()

y_train = training_set['toxic']
X_train = vectorizer.fit_transform(training_set['comment_text'])

print("y_train.shape: " + str(y_train.shape))
print("X_train.shape: " + str(X_train.shape))


y_train.shape: (15282,)
X_train.shape: (15282, 39767)


## Addestramento del Modello

In [4]:
#Verifica presenza della GPU
import tensorflow as tf

# Verifica il dispositivo attualmente utilizzato da TensorFlow
device_name = tf.test.gpu_device_name()

if device_name == '/device:GPU:0':
    print('GPU disponibile, TensorFlow sta utilizzando la GPU.')
else:
    print('GPU non disponibile, TensorFlow sta utilizzando la CPU.')


GPU disponibile, TensorFlow sta utilizzando la GPU.


2024-02-10 15:44:36.336784: I metal_plugin/src/device/metal_device.cc:1154] Metal device set to: Apple M1
2024-02-10 15:44:36.336825: I metal_plugin/src/device/metal_device.cc:296] systemMemory: 8.00 GB
2024-02-10 15:44:36.336847: I metal_plugin/src/device/metal_device.cc:313] maxCacheSize: 2.67 GB
2024-02-10 15:44:36.337354: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:306] Could not identify NUMA node of platform GPU ID 0, defaulting to 0. Your kernel may not have been built with NUMA support.
2024-02-10 15:44:36.337440: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:272] Created TensorFlow device (/device:GPU:0 with 0 MB memory) -> physical PluggableDevice (device: 0, name: METAL, pci bus id: <undefined>)


In [6]:
X_train = X_train.toarray()

In [5]:
model_filename = 'nn_classifier.pkl'
model = None

In [44]:
# Building the CNN Model
model = tf.keras.models.Sequential([
tf.keras.layers.Reshape((39767, 1), input_shape=(39767,)),  
tf.keras.layers.Conv1D(filters=25, kernel_size=11, strides=4,
activation='relu',input_shape=(39767, 1)),
tf.keras.layers.MaxPool1D(pool_size=3, strides=2,padding='same'),
tf.keras.layers.Conv1D(filters=65, kernel_size=5, padding='same',
activation='relu'),
tf.keras.layers.MaxPool1D(pool_size=3, strides=2,padding='same'),
tf.keras.layers.Conv1D(filters=70, kernel_size=3, padding='same',
activation='relu'),
tf.keras.layers.Conv1D(filters=60, kernel_size=1, padding='same',
activation='relu'),
tf.keras.layers.Dense(155, activation='relu'),
tf.keras.layers.Dropout(0.5),
tf.keras.layers.Dense(40, activation='relu'),
tf.keras.layers.Dropout(0.5),
tf.keras.layers.Dense(1,activation='sigmoid')
])

In [40]:
model.build()
model.compile(optimizer='adam',loss='binary_crossentropy',metrics=['accuracy'])
model.summary()

Model: "sequential_11"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 reshape_12 (Reshape)        (None, 39767, 1)          0         
                                                                 
 conv1d_58 (Conv1D)          (None, 9940, 25)          300       
                                                                 
 max_pooling1d_17 (MaxPooli  (None, 4970, 25)          0         
 ng1D)                                                           
                                                                 
 conv1d_59 (Conv1D)          (None, 4970, 65)          8190      
                                                                 
 max_pooling1d_18 (MaxPooli  (None, 2485, 65)          0         
 ng1D)                                                           
                                                                 
 conv1d_60 (Conv1D)          (None, 2485, 70)        

In [41]:
model.fit(X_train, y_train, epochs=10, batch_size=32)

Epoch 1/10
478/478 [==============================] - 88s 170ms/step - loss: 0.5179 - accuracy: 0.7998
Epoch 2/10
478/478 [==============================] - 80s 167ms/step - loss: 0.5086 - accuracy: 0.7999
Epoch 3/10
478/478 [==============================] - 79s 165ms/step - loss: 0.5077 - accuracy: 0.7999
Epoch 4/10
478/478 [==============================] - 82s 171ms/step - loss: 0.5061 - accuracy: 0.7999
Epoch 5/10
478/478 [==============================] - 92s 193ms/step - loss: 0.5053 - accuracy: 0.7999
Epoch 6/10
478/478 [==============================] - 103s 215ms/step - loss: 0.5041 - accuracy: 0.7999
Epoch 7/10
478/478 [==============================] - 101s 210ms/step - loss: 0.5034 - accuracy: 0.7999
Epoch 8/10
478/478 [==============================] - 100s 209ms/step - loss: 0.5030 - accuracy: 0.7999
Epoch 9/10
478/478 [==============================] - 101s 210ms/step - loss: 0.5027 - accuracy: 0.7999
Epoch 10/10
478/478 [==============================] - 93s 194ms/step

In [45]:
with open(model_filename, 'wb') as f:
    pickle.dump(model, f)

In [7]:
with open(model_filename, 'rb') as f:
    model = pickle.load(f)

2024-02-10 15:57:49.973128: I metal_plugin/src/device/metal_device.cc:1154] Metal device set to: Apple M1
2024-02-10 15:57:49.973174: I metal_plugin/src/device/metal_device.cc:296] systemMemory: 8.00 GB
2024-02-10 15:57:49.973181: I metal_plugin/src/device/metal_device.cc:313] maxCacheSize: 2.67 GB
2024-02-10 15:57:49.973264: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:306] Could not identify NUMA node of platform GPU ID 0, defaulting to 0. Your kernel may not have been built with NUMA support.
2024-02-10 15:57:49.973331: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:272] Created TensorFlow device (/job:localhost/replica:0/task:0/device:GPU:0 with 0 MB memory) -> physical PluggableDevice (device: 0, name: METAL, pci bus id: <undefined>)


# Testing del Sistema, Dataset "Non Lemmatizzato"

In [8]:
test_data = pd.read_csv("./../../datasets/test_set.csv")
test_data.dropna(inplace=True)
test_set = test_data[test_data['toxic'] != -1]
X_test = vectorizer.transform(test_set['comment_text'])
print(X_test.shape)



(63842, 39767)


In [9]:
X_test = X_test.toarray()
print(X_test.shape)


(63842, 39767)


In [10]:
y_test = test_set['toxic']
print("y_test.shape: " + str(y_test.shape))
y_test_sampled = y_test[:20000]

y_test.shape: (63842,)


In [11]:
# Supponendo che X_test sia il tuo dataset di dimensioni (63842, 39767)
# Reshape per adattare la forma di input del modello
X_test_reshaped = np.reshape(X_test, (X_test.shape[0],X_test.shape[1],1)) 

In [12]:
print(X_test_reshaped.shape)

(63842, 39767, 1)


In [19]:
X_test_sampled = X_test_reshaped[:20000]

In [1]:
 # Ora puoi fare le predizioni
y_pred = model.predict(X_test_sampled) # Le predizioni saranno di dimensioni (63842, 1)

NameError: name 'model' is not defined

In [34]:
y_pred = np.reshape(y_pred, (y_test_sampled.shape[0])) 

ValueError: cannot reshape array of size 49700000 into shape (20000,)

In [30]:
y_test_sampled.shape

(20000,)

In [23]:
y_pred_binary = np.where(y_pred > 0.5, 1, 0)

#Metriche: Accuracy,Precision,Recall
print("Accuracy: " + str(accuracy_score(y_test_sampled, y_pred_binary)))
print("Precision: " + str(precision_score(y_test_sampled, y_pred_binary)))
print("Recall: " + str(recall_score(y_test_sampled, y_pred_binary)))

ValueError: Classification metrics can't handle a mix of binary and unknown targets

In [ ]:
conf_matrix = confusion_matrix(y_test, y_pred_binary)

# Definisci le etichette delle classi
classes = ['Classe Negativa', 'Classe Positiva']

# Plotta la matrice di confusione
plt.figure(figsize=(6, 5))
sns.heatmap(conf_matrix, annot=True, fmt="d", cmap="Blues", xticklabels=classes, yticklabels=classes)
plt.xlabel('Predicted')
plt.ylabel('True')
plt.title('Matrice di Confusione')
plt.show()

# Neural Network, Dataset "Lemmatizzato"

In [ ]:
training_set_lem = pd.read_csv("./../../datasets/training_set_lemmatized.csv")
test_data_lem = pd.read_csv("./../../datasets/test_set_lemmatized.csv")
test_set_lem = test_data[test_data['toxic']!=-1]
test_data_lem.dropna(inplace=True)

In [ ]:
# Vettorizzazione con TF-IDF
vectorizer = TfidfVectorizer()
vectorizer_lem = TfidfVectorizer()

y_train = training_set['toxic']
X_train_lem = vectorizer_lem.fit_transform(training_set_lem['comment_text'])
X_test_lem = vectorizer_lem.transform(test_set_lem['comment_text'])

print("y_train.shape: " + str(y_train.shape))
print("X_train_lem.shape: " + str(X_train_lem.shape))
print("X_test_lem.shape: " + str(X_test_lem.shape))


## Addestramento del Modello

In [ ]:
#Verifica presenza della GPU
import tensorflow as tf

# Verifica il dispositivo attualmente utilizzato da TensorFlow
device_name = tf.test.gpu_device_name()

if device_name == '/device:GPU:0':
    print('GPU disponibile, TensorFlow sta utilizzando la GPU.')
else:
    print('GPU non disponibile, TensorFlow sta utilizzando la CPU.')

In [ ]:
X_train_lem = X_train_lem.toarray()

In [ ]:
# Building the CNN Model

model = tf.keras.models.Sequential([
tf.keras.layers.Reshape((34238, 1), input_shape=(34238,)),  
tf.keras.layers.Conv1D(filters=96, kernel_size=11, strides=4,
activation='relu',input_shape=(39767, 1)),
tf.keras.layers.Dropout(0.5),
tf.keras.layers.Conv1D(filters=120, kernel_size=5, padding='same',
activation='relu'),
tf.keras.layers.Conv1D(filters=240, kernel_size=3, padding='same',
activation='relu'),
tf.keras.layers.Dropout(0.5),
tf.keras.layers.Conv1D(filters=300, kernel_size=3, padding='same',
activation='relu'),
tf.keras.layers.Conv1D(filters=150, kernel_size=1, padding='same',
activation='relu'),
tf.keras.layers.MaxPool1D(pool_size=3, strides=2,padding='same'),
tf.keras.layers.Dense(80, activation='relu'),
tf.keras.layers.Dropout(0.5),
tf.keras.layers.Dense(40, activation='relu'),
tf.keras.layers.Dropout(0.5),
tf.keras.layers.Dense(1,activation='sigmoid')
])

In [ ]:
model.build()
model.compile(optimizer='adam',loss='binary_crossentropy',metrics=['accuracy'])
model.summary()

In [ ]:
model.fit(X_train_lem, y_train, epochs=10, batch_size=32)

In [ ]:
with open(model_filename, 'wb') as f:
    pickle.dump(model, f)

In [ ]:
with open(model_filename, 'rb') as f:
    model = pickle.load(f)

# Testing del Sistema, Dataset "Non Lemmatizzato"